In [14]:
# Importing important functions and packages in Python used in our analysis
import pandas as pd
import numpy as np
import tensorflow as tf
import random
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

# Set random seeds for reproducibility
random.seed(15)
np.random.seed(15)
tf.random.set_seed(15)

test


In [ ]:
# Load the original and transformed FRED-MD dataset, which is the same as the original study
data = pd.read_csv('/Users/d/Desktop/Code Blind 2/Data/2020-05.csv', header=None)
transformed_data = pd.read_csv("/Users/d/Desktop/Code Blind 2/Data_Out/Transformed.csv", header=None)   # Apply the necessary transformations as described in the paper
transformed_data = transformed_data.fillna(transformed_data.mean())

X = transformed_data.iloc[:, 1:]  # Exclude the target variable columns
X = (X - X.mean()) / X.std()  # Standardize the predictor variables

# Select the target variables (eight macroeconomic indicators, each from each economic aspect)
target_variables = [0, 22, 47, 2, 63, 77, 105]
y = transformed_data[target_variables]

# Defining the horizons, encoding dimension, epochs and batch size used for the autoencoders
horizons = [1,3,6,9,12,18,24]
encoding_dim = 1
epochs = 100
batch_size = 32

results = pd.DataFrame(index=horizons, columns=target_variables)

In [ ]:
# Create the autoencoder model
def create_autoencoder(input_dim, encoding_dim):
    # Define the input layer with the specified input dimension from this function
    input_layer = tf.keras.layers.Input(shape=(input_dim,))
    
    # Create the encoder layer with specified encoding dimension and regularization, using RELU activation function
    encoded = tf.keras.layers.Dense(encoding_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(input_layer)
    # Add dropout for regularization
    encoded = tf.keras.layers.Dropout(0.2)(encoded)
    
    # Create the decoder layer to reconstruct the input using theSigmoid activation function
    decoded = tf.keras.layers.Dense(input_dim, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l2(0.001))(encoded)
    
    # Define the autoencoder model which maps input to its reconstruction, as defined above
    autoencoder = tf.keras.models.Model(input_layer, decoded)
    # Define the encoder model maping input to its encoded representation
    encoder = tf.keras.models.Model(input_layer, encoded)
    
    return autoencoder, encoder

# Function to train  autoencoder
def train_autoencoder(X, encoding_dim, epochs, batch_size):
    # Determine the input dimension from the dataset
    input_dim = X.shape[1]
    # Create the autoencoder and encoder models
    autoencoder, encoder = create_autoencoder(input_dim, encoding_dim)
    
    # Compile the autoencoder model with Adam optimizer and MSE loss
    autoencoder.compile(optimizer='adam', loss='mse')
    # Set up early stopping to prevent overfitting
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
    
    # Train the autoencoder on the input data
    autoencoder.fit(X, X, epochs=epochs, batch_size=batch_size, shuffle=False, callbacks=[early_stopping])
    
    return encoder

# Function to forecast using the trained autoencoder
def forecast_with_autoencoder(X, y, horizon, encoding_dim, epochs, batch_size):
    # Define the window size based on the horizon
    window_size = 120 - horizon
    # Set up time series cross-validation with 5 splits, this number is used to balance the bias-variance tradeoff
    tscv = TimeSeriesSplit(n_splits=5)
    
    # List to store the mean squared prediction error for each split
    mspe_scores = []

    # Perform cross-validation by making use of the TimeSeriesSplit function in Python
    for train_index, test_index in tscv.split(X):
        # Split data into training and testing sets
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Train the autoencoder and obtain the encoder
        encoder = train_autoencoder(X_train, encoding_dim, epochs, batch_size)
        # Encode the training and testing data
        X_train_encoded = encoder.predict(X_train)
        X_test_encoded = encoder.predict(X_test)
        
        # Defining model for forecasting
        model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(64, activation='relu', input_shape=(encoding_dim,), kernel_regularizer=tf.keras.regularizers.l2(0.001)),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(1)
        ])
        # Runs the model with Adam optimizer and MSPE loss
        model.compile(optimizer='adam', loss='mse')
        # Set up early stopping for the  model
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
        
        # Train the model on the encoded training data
        model.fit(X_train_encoded, y_train, epochs=epochs, batch_size=batch_size, callbacks=[early_stopping])
        
        # Predict the target variable for the testing data
        y_pred = model.predict(X_test_encoded)
        
        # Calculate mean squared prediction error and append to the list
        mspe = np.mean((y_test - y_pred) ** 2)
        mspe_scores.append(mspe)
    
    # Return the average mean squared prediction error across all splits
    return np.mean(mspe_scores)


In [ ]:
# For-Looping through each horizon value
for horizon in horizons:

    # Initialize list to store MSPE values for the current horizon
    mspe_vector = []
    
    # Looping through each target variable
    for target_variable in target_variables:
        
        # Forecast using the autoencoder and calculate the MSPE
        mspe = forecast_with_autoencoder(X.values, y[target_variable].values.reshape(-1, 1), horizon, encoding_dim, epochs, batch_size)
        
        # Add the MSPE value to the vector defined above
        mspe_vector.append(mspe)
    
    # Store the MSPE vector in the results dataFrame with the current horizon as the index
    results.loc[horizon] = mspe_vector

# Prints the final results DataFrame of m
print(results)


In [ ]:
PCA_MSPEs = pd.read_csv('/Users/d/Desktop/Code Blind 2/Data_Out/PCA_MSPEs.csv', header=None)
# Relative_MSPEs_AE = results / PCA_MSPEs.iloc[:, [6]]
Relative_MSPEs_AE = results / PCA_MSPEs
print(Relative_MSPEs_AE)
